<a href="https://colab.research.google.com/github/yeonghun00/dc/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime

In [2]:
def view_page(url, gall_num):
  try:
    url = url + str(gall_num)
    result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
    bs_obj = BeautifulSoup(result.content, "html.parser")

    gall_num = gall_num
    title = bs_obj.find_all('span', {'class': 'title_subject'})[0].text
    content = bs_obj.find_all('div', {'class': 'write_div'})[0].text

    good = int(bs_obj.find_all('p', {'class': 'up_num'})[0].text)
    bad = int(bs_obj.find_all('p', {'class': 'down_num'})[0].text)

    date = bs_obj.find_all('span', {'class': 'gall_date'})[0].text
    views = bs_obj.find_all('span', {'class': 'gall_count'})[0].text
    comments = bs_obj.find_all('span', {'class': 'gall_comment'})[0].text

    date = datetime.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
    views = int(''.join(list(filter(str.isdigit, views))))
    comments = int(''.join(list(filter(str.isdigit, comments))))

    return [gall_num, date, title, content, good, bad, views, comments]
  except:
    pass

In [3]:
from multiprocessing import Pool

url = 'https://gall.dcinside.com/mgallery/board/view/?id=kospi&no='
current_id = 2184567
num = 50

with Pool() as p:
    galls = p.starmap(view_page, list(zip([url]*num, [x for x in range(current_id - num, current_id)])))

In [4]:
import pandas as pd

df = pd.DataFrame([x for x in galls if x!=None], columns=['id', 'datetime', 'title', 'content', 'good', 'bad', 'views', 'comments'])

In [5]:
df['content'] = [x.replace('\n',' ').replace('- dc official App', '') for x in df['content']]

In [6]:
df

,id,datetime,title,content,good,bad,views,comments
0,2184517,2022-06-29 16:13:28,원전주 세력새끼들 돌림빵 존나 놓네?,아주 돌아가면서 쏴올리면 개미들이 얼씨구나 하고 달려들줄 아나 요즘같은 장에뭐 특...,0,0,27,0
1,2184518,2022-06-29 16:13:29,나의 말일 투자 전략,오늘 곱버스 매도후 종가에 다시 레버리지 매수했음 내일 전반기 마지막 날이라 기...,0,0,29,0
2,2184519,2022-06-29 16:13:31,ㅋㅋ 노을 종배 성공 ㅋㅋ,시간외 개꿀 ~,0,0,20,0
3,2184520,2022-06-29 16:13:48,나랑 사귈사람 있냐? ㅡㅡ,ㅡㅡ,0,0,29,2
4,2184521,2022-06-29 16:13:55,윤석열 급부상할때부터 대통령할줄 알았다,보수가 대선 이기는건 이미 시작도 전에 확정이었으니까후보가 중요했는데 주식쟁이의 ...,0,0,26,0
5,2184522,2022-06-29 16:14:07,지금 좇고수들은 반도체 it담는거 아놐ㅋㅋㅋ컄ㅋㅋㅋㅋㅋㅋ,나도 모름ㅋㅋㅋㅋㅋㅋㅋㅋ컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1,0,67,2
6,2184523,2022-06-29 16:14:15,곡물 또 튈거같다,걍그렇다구,0,0,52,3
7,2184524,2022-06-29 16:14:19,환율보니 한국 투심이무너졌네,준비 단단히해라버스보다는 외환매수로 대응계좌보다는 현찰로보유,1,0,68,1
8,2184525,2022-06-29 16:14:20,일본은 기초과학은 튼튼하지만 한국이 더 대단하다고 본다,"반도체, 2차전지, 전기차 미래 산업은 한국이 점령하고 있는거 아니노 ㄷㄷ인구 감...",1,0,27,1
9,2184526,2022-06-29 16:15:21,와 카뱅 매도 리포트 나왔네,소신보소,0,0,90,3
